In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.12.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\jihyu\anaconda3\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 0.9278, train_loss: 2.04125118, train_accuracy: 0.4219, test_Accuracy: 0.1908
Epoch: [ 0] [    1/  468] time: 1.2805, train_loss: 1.90049446, train_accuracy: 0.4453, test_Accuracy: 0.3627
Epoch: [ 0] [    2/  468] time: 1.6432, train_loss: 1.63121724, train_accuracy: 0.5625, test_Accuracy: 0.4286
Epoch: [ 0] [    3/  468] time: 1.9863, train_loss: 1.39105177, train_accuracy: 0.6172, test_Accuracy: 0.5590
Epoch: [ 0] [    4/  468] time: 2.3565, train_loss: 1.17269945, train_accuracy: 0.7656, test_Accuracy: 0.6554
Epoch: [ 0] [    5/  468] time: 2.7088, train_loss: 0.78147626, train_accuracy: 0.8359, test_Accuracy: 0.6720
Epoch: [ 0] [    6/  468] time: 3.0462, train_loss: 0.84694874, train_accuracy: 0.7656, test_Accuracy: 0.7493
Epoch: [ 0] [    7/  468] time: 3.3862, train_loss: 0.70497561, train_accuracy: 0.8125, test_Accuracy: 0.7753
Epoch: [ 0] [    8/  468] time: 3.7265, train_loss: 0.67902446, train_accuracy: 0.7812, test_Accuracy: 0.7119
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 27.9131, train_loss: 0.26206583, train_accuracy: 0.9297, test_Accuracy: 0.9274
Epoch: [ 0] [   76/  468] time: 28.2827, train_loss: 0.31665605, train_accuracy: 0.9219, test_Accuracy: 0.9261
Epoch: [ 0] [   77/  468] time: 28.6419, train_loss: 0.16011655, train_accuracy: 0.9297, test_Accuracy: 0.9269
Epoch: [ 0] [   78/  468] time: 29.0079, train_loss: 0.25994408, train_accuracy: 0.9297, test_Accuracy: 0.9271
Epoch: [ 0] [   79/  468] time: 29.3926, train_loss: 0.27042767, train_accuracy: 0.9141, test_Accuracy: 0.9333
Epoch: [ 0] [   80/  468] time: 29.7699, train_loss: 0.15918685, train_accuracy: 0.9531, test_Accuracy: 0.9374
Epoch: [ 0] [   81/  468] time: 30.1371, train_loss: 0.20277016, train_accuracy: 0.9453, test_Accuracy: 0.9371
Epoch: [ 0] [   82/  468] time: 30.5168, train_loss: 0.15390113, train_accuracy: 0.9688, test_Accuracy: 0.9352
Epoch: [ 0] [   83/  468] time: 30.8799, train_loss: 0.25157678, train_accuracy: 0.9453, test_Accuracy: 0.9327
E

Epoch: [ 0] [  149/  468] time: 56.5170, train_loss: 0.15858211, train_accuracy: 0.9531, test_Accuracy: 0.9491
Epoch: [ 0] [  150/  468] time: 56.9098, train_loss: 0.13398074, train_accuracy: 0.9453, test_Accuracy: 0.9462
Epoch: [ 0] [  151/  468] time: 57.2993, train_loss: 0.11849385, train_accuracy: 0.9766, test_Accuracy: 0.9485
Epoch: [ 0] [  152/  468] time: 57.7005, train_loss: 0.16968423, train_accuracy: 0.9531, test_Accuracy: 0.9502
Epoch: [ 0] [  153/  468] time: 58.0871, train_loss: 0.08752830, train_accuracy: 0.9766, test_Accuracy: 0.9521
Epoch: [ 0] [  154/  468] time: 58.4746, train_loss: 0.06086129, train_accuracy: 0.9688, test_Accuracy: 0.9519
Epoch: [ 0] [  155/  468] time: 58.8834, train_loss: 0.06780367, train_accuracy: 0.9844, test_Accuracy: 0.9488
Epoch: [ 0] [  156/  468] time: 59.2901, train_loss: 0.10757461, train_accuracy: 0.9688, test_Accuracy: 0.9458
Epoch: [ 0] [  157/  468] time: 59.7392, train_loss: 0.21850777, train_accuracy: 0.9609, test_Accuracy: 0.9427
E

Epoch: [ 0] [  223/  468] time: 86.2382, train_loss: 0.11946664, train_accuracy: 0.9609, test_Accuracy: 0.9588
Epoch: [ 0] [  224/  468] time: 86.6291, train_loss: 0.08507390, train_accuracy: 0.9609, test_Accuracy: 0.9617
Epoch: [ 0] [  225/  468] time: 87.0268, train_loss: 0.06616100, train_accuracy: 0.9922, test_Accuracy: 0.9620
Epoch: [ 0] [  226/  468] time: 87.4318, train_loss: 0.11150223, train_accuracy: 0.9609, test_Accuracy: 0.9594
Epoch: [ 0] [  227/  468] time: 87.8381, train_loss: 0.10491222, train_accuracy: 0.9609, test_Accuracy: 0.9569
Epoch: [ 0] [  228/  468] time: 88.2668, train_loss: 0.09345613, train_accuracy: 0.9531, test_Accuracy: 0.9538
Epoch: [ 0] [  229/  468] time: 88.6623, train_loss: 0.12063843, train_accuracy: 0.9766, test_Accuracy: 0.9526
Epoch: [ 0] [  230/  468] time: 89.0673, train_loss: 0.07478899, train_accuracy: 0.9844, test_Accuracy: 0.9529
Epoch: [ 0] [  231/  468] time: 89.4817, train_loss: 0.10161912, train_accuracy: 0.9688, test_Accuracy: 0.9579
E

Epoch: [ 0] [  297/  468] time: 116.0432, train_loss: 0.09167143, train_accuracy: 0.9609, test_Accuracy: 0.9548
Epoch: [ 0] [  298/  468] time: 116.4502, train_loss: 0.07951744, train_accuracy: 0.9609, test_Accuracy: 0.9559
Epoch: [ 0] [  299/  468] time: 116.8514, train_loss: 0.07619283, train_accuracy: 0.9844, test_Accuracy: 0.9564
Epoch: [ 0] [  300/  468] time: 117.2751, train_loss: 0.03148454, train_accuracy: 1.0000, test_Accuracy: 0.9586
Epoch: [ 0] [  301/  468] time: 117.6729, train_loss: 0.11177504, train_accuracy: 0.9609, test_Accuracy: 0.9604
Epoch: [ 0] [  302/  468] time: 118.0929, train_loss: 0.24913277, train_accuracy: 0.9375, test_Accuracy: 0.9624
Epoch: [ 0] [  303/  468] time: 118.4991, train_loss: 0.14715631, train_accuracy: 0.9531, test_Accuracy: 0.9609
Epoch: [ 0] [  304/  468] time: 118.8971, train_loss: 0.08163442, train_accuracy: 0.9688, test_Accuracy: 0.9573
Epoch: [ 0] [  305/  468] time: 119.2961, train_loss: 0.03516765, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  371/  468] time: 146.1747, train_loss: 0.12156620, train_accuracy: 0.9375, test_Accuracy: 0.9641
Epoch: [ 0] [  372/  468] time: 146.5679, train_loss: 0.11637696, train_accuracy: 0.9688, test_Accuracy: 0.9632
Epoch: [ 0] [  373/  468] time: 146.9708, train_loss: 0.09501597, train_accuracy: 0.9766, test_Accuracy: 0.9629
Epoch: [ 0] [  374/  468] time: 147.3981, train_loss: 0.04967953, train_accuracy: 0.9922, test_Accuracy: 0.9624
Epoch: [ 0] [  375/  468] time: 147.7943, train_loss: 0.11038041, train_accuracy: 0.9609, test_Accuracy: 0.9632
Epoch: [ 0] [  376/  468] time: 148.1925, train_loss: 0.05545717, train_accuracy: 0.9766, test_Accuracy: 0.9640
Epoch: [ 0] [  377/  468] time: 148.6382, train_loss: 0.05020961, train_accuracy: 0.9844, test_Accuracy: 0.9654
Epoch: [ 0] [  378/  468] time: 149.1408, train_loss: 0.06328631, train_accuracy: 0.9766, test_Accuracy: 0.9666
Epoch: [ 0] [  379/  468] time: 149.6584, train_loss: 0.10655840, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  445/  468] time: 177.9388, train_loss: 0.16332448, train_accuracy: 0.9609, test_Accuracy: 0.9650
Epoch: [ 0] [  446/  468] time: 178.3462, train_loss: 0.05624708, train_accuracy: 0.9844, test_Accuracy: 0.9654
Epoch: [ 0] [  447/  468] time: 178.7467, train_loss: 0.06274444, train_accuracy: 0.9766, test_Accuracy: 0.9697
Epoch: [ 0] [  448/  468] time: 179.1701, train_loss: 0.18295017, train_accuracy: 0.9453, test_Accuracy: 0.9700
Epoch: [ 0] [  449/  468] time: 179.5726, train_loss: 0.05599635, train_accuracy: 0.9922, test_Accuracy: 0.9703
Epoch: [ 0] [  450/  468] time: 179.9831, train_loss: 0.07327923, train_accuracy: 0.9609, test_Accuracy: 0.9688
Epoch: [ 0] [  451/  468] time: 180.4287, train_loss: 0.03573425, train_accuracy: 1.0000, test_Accuracy: 0.9669
Epoch: [ 0] [  452/  468] time: 180.8481, train_loss: 0.12048012, train_accuracy: 0.9688, test_Accuracy: 0.9654
Epoch: [ 0] [  453/  468] time: 181.2542, train_loss: 0.04907373, train_accuracy: 0.9844, test_Accuracy: